## LOAD PDF FILES

In [1]:
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader,UnstructuredPDFLoader


c:\Users\vigna\Desktop\RAG-KN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
### PyPDFLoader
print("PyPDFLoader")
try:
    pypdf_loader = PyPDFLoader("data/pdf/attention.pdf")
    pypdf_docs = pypdf_loader.load()
    print(pypdf_docs[0].page_content)
except Exception as e:
    print(f"Error : {e}")

PyPDFLoader
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions


In [6]:
### PyMuPDFLoader
print("PyMuPDFLoader")
try:
    pymupdf_loader = PyMuPDFLoader("data/pdf/attention.pdf")
    pymupdf_docs = pypdf_loader.load()
    print(pymupdf_docs[0].page_content)
except Exception as e:
    print(f"Error : {e}")

PyMuPDFLoader
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolution

In [8]:
# 📊 PDF Loader Comparison
print("\n📊 PDF Loader Comparison:")
print("\nPyPDFLoader:")
print("  ✅ Simple and reliable")
print("  ✅ Good for most PDFs")
print("  ✅ Preserves page numbers")
print("  ❌ Basic text extraction")
print("  Use when: Standard text PDFs")

print("\nPyMuPDFLoader:")
print("  ✅ Fast processing")
print("  ✅ Good text extraction")
print("  ✅ Image extraction support")
print("  Use when: Speed is important")


📊 PDF Loader Comparison:

PyPDFLoader:
  ✅ Simple and reliable
  ✅ Good for most PDFs
  ✅ Preserves page numbers
  ❌ Basic text extraction
  Use when: Standard text PDFs

PyMuPDFLoader:
  ✅ Fast processing
  ✅ Good text extraction
  ✅ Image extraction support
  Use when: Speed is important


### Handling PDF Challenges 
🎯 Purpose of This Section
PDFs are notoriously difficult to parse because they:

- Store text in complex ways (not just simple text)
- Can have formatting issues
- May contain scanned images (requiring OCR)
- Often have extraction artifacts


In [ ]:
# Example of raw PDF extraction
raw_pdf_text = """Company Financial Report


    The ﬁnancial performance for ﬁscal year 2024
    shows signiﬁcant growth in proﬁtability.
    
    
    
    Revenue increased by 25%.
    
The company's efﬁciency improved due to workﬂow
optimization.


Page 1 of 10
"""



# Apply the cleaning function
def clean_text(text):
    # Remove excessive whitespace
    text = " ".join(text.split())
    
    # Fix ligatures
    text = text.replace("ﬁ", "fi")
    text = text.replace("ﬂ", "fl")
    
    return text

cleaned = clean_text(raw_pdf_text)
print("BEFORE:")
print(repr(raw_pdf_text))
print("\nAFTER:")
print((cleaned))


BEFORE:
Company Financial Report


    The ﬁnancial performance for ﬁscal year 2024
    shows signiﬁcant growth in proﬁtability.



    Revenue increased by 25%.

The company's efﬁciency improved due to workﬂow
optimization.


Page 1 of 10


AFTER:
"Company Financial Report The financial performance for fiscal year 2024 shows significant growth in profitability. Revenue increased by 25%. The company's efficiency improved due to workflow optimization. Page 1 of 10"


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [18]:
from typing import List
from langchain_core.documents import Document


class SmartPDFProcessor:
    """ Advanced PDF Processing with error handling """
    def __init__(self,chunk_size=1000,chunk_overlap=100):
        self.chunk_size=chunk_size
        self.chunk_overlap=chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            separators=[" "]
        )
    def clean_text(self,text):
            # Remove excessive whitespace
            text = " ".join(text.split())
            
            # Fix ligatures
            text = text.replace("ﬁ", "fi")
            text = text.replace("ﬂ", "fl")
            
            return text
    def process_pdf(self,pdf_path:str)->List[Document]:
        """Process PDF with smart chunking and metadata enhancement """
        # Load PDF
        loader = PyPDFLoader(pdf_path)
        pages= loader.load()

        ## Process each page
        processed_chunks = []
        for page_num,page in enumerate(pages):
            ## Clean text 
            cleaned_text = self.clean_text(page.page_content)

            ## Skip nearly empty pages
            if len(cleaned_text.strip())<50:
                 continue 
                 
            ## Create chunks with enhanced metadata

            chunks = self.text_splitter.create_documents(
                texts=[cleaned_text],
                metadatas=[{
                     **page.metadata,
                     "page":page_num+1,
                     "total_pages":len(pages),
                     "chunk_method":"smart_pdf_processor",
                     "char_count":len(cleaned_text)

                }]
            )
            # print(chunks)
            processed_chunks.extend(chunks)
        return processed_chunks

            




In [20]:
processor = SmartPDFProcessor()

In [21]:
## Process a PDF If available
try:
    smart_chunks = processor.process_pdf("data/pdf/attention.pdf")
    print(f"processed into {len(smart_chunks)} smart chunks")

    # show enhanced chunks
    if smart_chunks:
        print("\n Sample chunk metadata: ")
        for key,value in smart_chunks[0].metadata.items():
            print(f"{key}:{value}")
except Exception as e:
    print(f"Processing Error : {e} ")

processed into 49 smart chunks

 Sample chunk metadata: 
producer:pdfTeX-1.40.25
creator:LaTeX with hyperref
creationdate:2024-04-10T21:11:43+00:00
author:
keywords:
moddate:2024-04-10T21:11:43+00:00
ptex.fullbanner:This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5
subject:
title:
trapped:/False
source:data/pdf/attention.pdf
total_pages:15
page:1
page_label:1
chunk_method:smart_pdf_processor
char_count:2858
